# Week 14: Connected & Cooperative Driving

### Topics Covered

- Vehicle-to-Everything (V2X) Communication (V2V, V2I); Utilizing cloud computing; Over-the-Air (OTA) updates

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Understand the key concepts
2. Implement algorithms
3. Apply techniques to real-world problems

---

## Setup

Import required libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle, FancyBboxPatch, Polygon, FancyArrowPatch
from matplotlib.collections import PatchCollection
from dataclasses import dataclass, field
from typing import List, Dict, Tuple, Optional
import time
from enum import Enum

# Set random seed for reproducibility
np.random.seed(42)

## 1. Vehicle-to-Everything (V2X) Communication

V2X enables vehicles to communicate with their environment, enhancing safety and efficiency beyond what sensors alone can achieve.

### V2X Communication Types

**V2V (Vehicle-to-Vehicle)**
- Direct communication between vehicles
- Share position, speed, heading, intentions
- Range: typically 300m
- Use cases: Collision avoidance, cooperative lane change, platooning

**V2I (Vehicle-to-Infrastructure)**
- Communication with traffic lights, signs, roadside units
- Receive traffic signal timing, speed advisories, road conditions
- Use cases: Green wave coordination, hazard warnings, parking availability

**V2P (Vehicle-to-Pedestrian)**
- Communicate with smartphones/wearables
- Pedestrian location and trajectory
- Use cases: Crosswalk safety, vulnerable road user protection

**V2N (Vehicle-to-Network)**
- Communication via cellular network (4G/5G)
- Cloud services, traffic management centers
- Use cases: Real-time traffic, HD map updates, remote diagnostics

**V2C (Vehicle-to-Cloud)**
- Upload sensor data, download software updates
- Fleet management, data aggregation
- Use cases: OTA updates, collective learning

### V2X Technology Standards

**DSRC (Dedicated Short-Range Communications)**
- Based on IEEE 802.11p (WiFi variant)
- Frequency: 5.9 GHz
- Latency: ~50ms
- Range: 300-1000m
- Status: Widely deployed in US (before 2020)

**C-V2X (Cellular V2X)**
- Based on 3GPP standards (LTE/5G)
- Two modes:
  - **Direct (sidelink)**: Vehicle-to-vehicle without network
  - **Network**: Via cellular infrastructure
- Latency: 20-100ms (LTE), <10ms (5G)
- Range: Up to several kilometers
- Status: Growing adoption, especially in China and Europe

**Comparison:**

| Feature | DSRC | C-V2X (LTE) | C-V2X (5G) |
|---------|------|-------------|------------|
| Latency | ~50ms | 20-100ms | <10ms |
| Range | 300-1000m | Several km | Several km |
| Throughput | 6-27 Mbps | 100+ Mbps | 1+ Gbps |
| Maturity | Mature | Mature | Emerging |
| Infrastructure | Dedicated | Uses cellular | Uses 5G |

### V2X Messages

**Basic Safety Message (BSM)** / **Cooperative Awareness Message (CAM)**
- Broadcast 10 times per second
- Contains: Position, speed, heading, acceleration, vehicle size
- Size: ~200 bytes

**Signal Phase and Timing (SPaT)**
- Traffic signal state and timing
- Current phase (red/yellow/green)
- Time until phase change

**MAP (Intersection Geometry)**
- Lane geometry, turn restrictions
- Static map of intersection

**Personal Safety Message (PSM)**
- Pedestrian/cyclist location
- Movement prediction

### Benefits of V2X

**Safety:**
- **See Through**: Know what's beyond sensor range or occlusions
- **Forward Collision Warning**: Alert about stopped vehicles ahead
- **Intersection Collision Avoidance**: Prevent red-light violations
- **Emergency Vehicle Approaching**: Clear path for ambulances

**Efficiency:**
- **Green Wave**: Time speed to hit all green lights
- **Eco-Routing**: Optimize for fuel efficiency
- **Platooning**: Vehicles travel in tight formation, reduce drag

**Comfort:**
- **Parking Availability**: Find open spots
- **Road Condition Updates**: Potholes, construction, weather

### Challenges

**1. Penetration Rate**
- V2X only effective when enough vehicles equipped
- Chicken-and-egg problem
- Estimated need: >20% for safety benefits

**2. Security**
- Spoofing: Fake messages cause accidents
- Privacy: Tracking vehicles via broadcast messages
- Solutions: PKI (Public Key Infrastructure), pseudonymous certificates

**3. Interoperability**
- Different standards (DSRC vs. C-V2X)
- Different regions (EU vs. US vs. China)

**4. Latency & Reliability**
- Safety-critical messages must be timely
- Network congestion can cause delays
- Required: <100ms latency, >99% reliability

**5. Regulatory**
- Spectrum allocation (US removed 5.9 GHz for DSRC in 2020)
- Privacy laws (GDPR, CCPA)
- Cybersecurity mandates (UNECE WP.29)

---

## 1.1 Cooperative Driving Applications

**Cooperative Adaptive Cruise Control (CACC)**
- Vehicles coordinate speeds via V2V
- Tighter spacing than sensor-based ACC
- Gap: 0.5-1.0 seconds vs. 1.5-2.0 seconds
- Fuel savings: 10-20% due to reduced aerodynamic drag

**Platooning**
- Multiple vehicles travel in tight formation
- Lead vehicle controls speed
- Followers maintain constant gap
- Applications: Truck platooning on highways

**Cooperative Lane Change**
- Vehicle broadcasts lane change intent
- Neighbors adjust to create gap
- Smoother, safer lane changes

**Intersection Management**
- Centralized coordinator (V2I)
- Slot-based reservation system
- Eliminate stop-and-go, improve throughput by 2-3x

In [ ]:
# Implementation: V2X Communication Simulator

class MessageType(Enum):
    """V2X message types"""
    BSM = "Basic Safety Message"
    SPAT = "Signal Phase and Timing"
    MAP = "MAP Data"
    PSM = "Personal Safety Message"

@dataclass
class V2XMessage:
    """V2X message structure"""
    message_type: MessageType
    sender_id: str
    timestamp: float
    position: Tuple[float, float]
    velocity: float = 0.0
    heading: float = 0.0
    acceleration: float = 0.0
    additional_data: Dict = field(default_factory=dict)

class V2XNetwork:
    """Simulates V2X communication network"""
    
    def __init__(self, max_range=300.0, latency=0.05, packet_loss=0.01):
        """
        Initialize V2X network
        
        Parameters:
        - max_range: Communication range in meters
        - latency: Message latency in seconds
        - packet_loss: Probability of message loss (0.0 to 1.0)
        """
        self.max_range = max_range
        self.latency = latency
        self.packet_loss = packet_loss
        self.message_buffer = []  # Messages in transit
        self.current_time = 0.0
    
    def broadcast_message(self, message: V2XMessage):
        """Broadcast message to all vehicles in range"""
        # Add latency
        delivery_time = self.current_time + self.latency
        
        # Simulate packet loss
        if np.random.random() > self.packet_loss:
            self.message_buffer.append((delivery_time, message))
    
    def receive_messages(self, receiver_position: Tuple[float, float]) -> List[V2XMessage]:
        """Receive all messages for a vehicle at given position"""
        self.current_time += 0.1  # Advance time
        
        received = []
        remaining = []
        
        for delivery_time, msg in self.message_buffer:
            # Check if message is ready for delivery
            if delivery_time <= self.current_time:
                # Check if receiver is in range
                distance = np.sqrt(
                    (receiver_position[0] - msg.position[0])**2 +
                    (receiver_position[1] - msg.position[1])**2
                )
                
                if distance <= self.max_range:
                    received.append(msg)
            else:
                remaining.append((delivery_time, msg))
        
        self.message_buffer = remaining
        return received

@dataclass
class ConnectedVehicle:
    """Vehicle with V2X capability"""
    id: str
    x: float
    y: float
    velocity: float
    heading: float
    target_velocity: float = 20.0
    
    def create_bsm(self, timestamp: float) -> V2XMessage:
        """Create Basic Safety Message"""
        return V2XMessage(
            message_type=MessageType.BSM,
            sender_id=self.id,
            timestamp=timestamp,
            position=(self.x, self.y),
            velocity=self.velocity,
            heading=self.heading,
            acceleration=0.0
        )
    
    def update_from_v2x(self, messages: List[V2XMessage], dt: float):
        """Update vehicle state based on V2X messages"""
        # Simple CACC: adjust speed based on vehicle ahead
        vehicles_ahead = []
        
        for msg in messages:
            if msg.message_type == MessageType.BSM and msg.sender_id != self.id:
                # Check if vehicle is ahead
                dx = msg.position[0] - self.x
                dy = msg.position[1] - self.y
                
                # Roughly ahead (same lane, within 30 degrees)
                angle_to_vehicle = np.arctan2(dy, dx)
                angle_diff = abs(angle_to_vehicle - self.heading)
                
                if dx > 0 and angle_diff < np.deg2rad(30):
                    distance = np.sqrt(dx**2 + dy**2)
                    vehicles_ahead.append((distance, msg.velocity))
        
        # CACC control logic
        if vehicles_ahead:
            # Find closest vehicle ahead
            closest_dist, closest_vel = min(vehicles_ahead, key=lambda x: x[0])
            
            # Desired gap: 1.0 second time gap
            desired_gap = self.velocity * 1.0 + 5.0  # 5m minimum
            
            # Simple proportional control
            gap_error = closest_dist - desired_gap
            speed_error = closest_vel - self.velocity
            
            # Acceleration command
            accel = 0.3 * gap_error + 0.5 * speed_error
        else:
            # No vehicle ahead, accelerate to target speed
            accel = 0.5 * (self.target_velocity - self.velocity)
        
        # Apply acceleration
        accel = np.clip(accel, -3.0, 2.0)
        self.velocity += accel * dt
        self.velocity = np.clip(self.velocity, 0, 30)
        
        # Update position
        self.x += self.velocity * np.cos(self.heading) * dt
        self.y += self.velocity * np.sin(self.heading) * dt


# Demonstration: CACC Platoon Simulation
def demonstrate_cacc():
    print("=== Cooperative Adaptive Cruise Control (CACC) Simulation ===\n")
    
    # Create V2X network
    network = V2XNetwork(max_range=300.0, latency=0.05, packet_loss=0.02)
    
    # Create platoon of 4 vehicles
    vehicles = [
        ConnectedVehicle(id="Lead", x=0, y=0, velocity=20, heading=0, target_velocity=20),
        ConnectedVehicle(id="V1", x=-15, y=0, velocity=20, heading=0, target_velocity=20),
        ConnectedVehicle(id="V2", x=-30, y=0, velocity=20, heading=0, target_velocity=20),
        ConnectedVehicle(id="V3", x=-45, y=0, velocity=20, heading=0, target_velocity=20),
    ]
    
    # Simulation parameters
    dt = 0.1
    duration = 30.0
    steps = int(duration / dt)
    
    # Lead vehicle deceleration scenario
    decel_start = 10.0
    decel_duration = 5.0
    
    # History tracking
    history = {v.id: {'time': [], 'x': [], 'velocity': [], 'gap': []} for v in vehicles}
    
    print("Scenario: Lead vehicle decelerates, platoon responds via V2X\n")
    
    for step in range(steps):
        time = step * dt
        
        # Lead vehicle behavior: decelerate after t=10s
        if decel_start <= time < decel_start + decel_duration:
            vehicles[0].target_velocity = 15.0
        else:
            vehicles[0].target_velocity = 20.0
        
        # Broadcast BSM from all vehicles
        for vehicle in vehicles:
            bsm = vehicle.create_bsm(time)
            network.broadcast_message(bsm)
        
        # Receive and process V2X messages
        for vehicle in vehicles:
            messages = network.receive_messages((vehicle.x, vehicle.y))
            vehicle.update_from_v2x(messages, dt)
        
        # Record history
        for i, vehicle in enumerate(vehicles):
            history[vehicle.id]['time'].append(time)
            history[vehicle.id]['x'].append(vehicle.x)
            history[vehicle.id]['velocity'].append(vehicle.velocity)
            
            # Calculate gap to vehicle ahead
            if i > 0:
                gap = vehicles[i-1].x - vehicle.x
            else:
                gap = 0
            history[vehicle.id]['gap'].append(gap)
    
    # Analyze results
    print("Results:")
    for vehicle in vehicles:
        if vehicle.id != "Lead":
            min_gap = min(history[vehicle.id]['gap'])
            print(f"  {vehicle.id}: Min gap to leader = {min_gap:.2f} m")
    
    # Visualize
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Position over time
    ax1 = axes[0, 0]
    colors = ['red', 'blue', 'green', 'orange']
    for vehicle, color in zip(vehicles, colors):
        ax1.plot(history[vehicle.id]['time'], history[vehicle.id]['x'], 
                linewidth=2, label=vehicle.id, color=color)
    ax1.set_xlabel('Time (s)', fontsize=11)
    ax1.set_ylabel('Position (m)', fontsize=11)
    ax1.set_title('Vehicle Positions', fontsize=13, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Velocity over time
    ax2 = axes[0, 1]
    for vehicle, color in zip(vehicles, colors):
        ax2.plot(history[vehicle.id]['time'], history[vehicle.id]['velocity'], 
                linewidth=2, label=vehicle.id, color=color)
    ax2.set_xlabel('Time (s)', fontsize=11)
    ax2.set_ylabel('Velocity (m/s)', fontsize=11)
    ax2.set_title('Vehicle Velocities', fontsize=13, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Gap to vehicle ahead
    ax3 = axes[1, 0]
    for i, (vehicle, color) in enumerate(zip(vehicles[1:], colors[1:]), 1):
        ax3.plot(history[vehicle.id]['time'], history[vehicle.id]['gap'], 
                linewidth=2, label=f"{vehicle.id} to {vehicles[i-1].id}", color=color)
    ax3.axhline(y=10, color='r', linestyle='--', linewidth=1.5, alpha=0.7, label='Min Safe Gap')
    ax3.set_xlabel('Time (s)', fontsize=11)
    ax3.set_ylabel('Gap (m)', fontsize=11)
    ax3.set_title('Inter-Vehicle Gaps', fontsize=13, fontweight='bold')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Platoon visualization at specific time
    ax4 = axes[1, 1]
    snapshot_time = 15.0
    snapshot_idx = int(snapshot_time / dt)
    
    ax4.set_xlim(-60, 400)
    ax4.set_ylim(-5, 5)
    ax4.set_aspect('equal')
    ax4.axhline(y=0, color='gray', linestyle='-', linewidth=1)
    
    for i, (vehicle, color) in enumerate(zip(vehicles, colors)):
        x_pos = history[vehicle.id]['x'][snapshot_idx]
        rect = Rectangle((x_pos-2.5, -1), 5, 2, facecolor=color, edgecolor='black', linewidth=2, alpha=0.7)
        ax4.add_patch(rect)
        ax4.text(x_pos, -3, vehicle.id, ha='center', fontsize=10, fontweight='bold')
        
        # Draw V2X communication range
        circle = Circle((x_pos, 0), network.max_range, fill=False, edgecolor=color, 
                       linestyle=':', linewidth=1.5, alpha=0.3)
        ax4.add_patch(circle)
    
    ax4.set_xlabel('Position (m)', fontsize=11)
    ax4.set_title(f'Platoon Configuration at t={snapshot_time}s', fontsize=13, fontweight='bold')
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n" + "="*50)
    print("✓ CACC maintains safe gaps via V2X communication")
    print("="*50)

demonstrate_cacc()

---

## 2. Cloud Computing for Autonomous Vehicles

Cloud computing extends the capabilities of autonomous vehicles beyond their onboard computational resources.

### Architecture

**Three-Tier Computing:**

1. **Vehicle Edge** (Local)
   - Real-time sensor processing
   - Immediate safety decisions
   - Latency: <10ms
   - Examples: Object detection, path planning

2. **Edge/Fog Nodes** (Roadside)
   - Intermediate computation at roadside units
   - Aggregation from multiple vehicles
   - Latency: 10-50ms
   - Examples: Intersection management, local traffic optimization

3. **Cloud** (Remote)
   - Massive computation and storage
   - Non-real-time tasks
   - Latency: 100-500ms
   - Examples: HD map updates, fleet learning, diagnostics

### Use Cases

**HD Map Updates**
- Vehicles upload sensor data (cameras, LiDAR)
- Cloud aggregates and processes to update maps
- Detects: Road changes, construction, new signs
- Distributes updated maps to fleet

**Fleet Learning**
- Vehicles encounter edge cases (unusual scenarios)
- Upload anonymized data to cloud
- Cloud trains improved models
- Distribute updates via OTA

**Computation Offloading**
- Resource-intensive tasks sent to cloud
- Examples: Route optimization with traffic, parking search
- Trade-off: Latency vs. computational power

**Remote Monitoring & Diagnostics**
- Continuous upload of vehicle health metrics
- Predictive maintenance (detect failures before they occur)
- Examples: Brake wear, battery degradation, sensor calibration

**Simulation & Validation**
- Upload real-world scenarios to cloud
- Replay in simulation for testing
- Validate software updates before deployment

### Edge vs. Cloud Trade-offs

| Criterion | Vehicle Edge | Cloud |
|-----------|--------------|-------|
| Latency | <10ms | 100-500ms |
| Computation | Limited (GPUs) | Unlimited (datacenters) |
| Data Storage | ~TB | Exabytes |
| Connectivity | Always available | Requires network |
| Privacy | Data stays local | Data transmitted |
| Cost | Hardware cost | Subscription cost |

**Decision Factors:**
- **Safety-critical** → Must be on-vehicle (cannot rely on network)
- **Real-time** → Edge (latency-sensitive)
- **Compute-intensive** → Cloud (if latency permits)
- **Privacy-sensitive** → Edge (avoid data transmission)
- **Fleet-wide** → Cloud (aggregation needed)

### Challenges

**1. Connectivity**
- Network coverage gaps (tunnels, rural areas)
- Handoffs between cell towers
- Solution: Graceful degradation when disconnected

**2. Bandwidth**
- Sensor data is massive (1GB per minute for cameras + LiDAR)
- Cannot upload everything in real-time
- Solution: Selective upload (interesting scenarios only)

**3. Latency**
- Cloud round-trip too slow for safety-critical tasks
- 5G promises <10ms, but not always achieved
- Solution: Hybrid edge-cloud architecture

**4. Cost**
- Data transmission costs (cellular)
- Cloud compute/storage costs
- Solution: Optimize what to upload/offload

**5. Security**
- Man-in-the-middle attacks
- Data tampering
- Solution: Encryption (TLS), authentication, integrity checks

---

## 2.1 Cloud-Based HD Map Updates

**Problem:** HD maps become outdated due to road changes.

**Solution:**
1. Vehicles detect map discrepancies via sensor fusion
2. Upload change reports to cloud with sensor data
3. Cloud aggregates reports from multiple vehicles
4. Human verification or automated processing
5. Update map database
6. Distribute incremental updates to fleet

**Example Pipeline:**
```
Vehicle A: "Construction detected at (lat, lon)"
Vehicle B: "Lane closure at (lat, lon)"  [same location]
Vehicle C: "Cone detected at (lat, lon)"  [same location]
  ↓
Cloud: "High confidence: Construction at (lat, lon)"
  ↓
Update HD Map: Mark temporary lane closure
  ↓
Distribute to fleet via OTA
```

**Benefits:**
- **Crowdsourced**: Fleet collectively maintains maps
- **Always up-to-date**: Changes detected within hours
- **Cost-effective**: Cheaper than manual surveying

---

## 3. Over-the-Air (OTA) Software Updates

OTA enables remote software updates without requiring physical service visits.

### Why OTA for Autonomous Vehicles?

**Traditional automotive:**
- Software updates require dealer visit
- Expensive, inconvenient
- Updates rare (maybe once per year)

**Autonomous vehicles:**
- Software is critical to safety
- Need rapid bug fixes and improvements
- OTA is essential for scalability

### OTA Architecture

**Components:**

1. **OTA Server** (Cloud)
   - Stores software packages
   - Manages update campaigns
   - Tracks fleet update status

2. **Vehicle Gateway**
   - Downloads updates
   - Verifies integrity (cryptographic signatures)
   - Installs updates
   - Rollback if installation fails

3. **Telematics Control Unit (TCU)**
   - Manages cellular connectivity
   - Coordinates update download

### Update Types

**A/B Partition Updates**
- Two system partitions: A (active) and B (backup)
- Update installed to inactive partition
- Reboot to switch partitions
- If update fails, rollback to previous partition
- Used by: Tesla, most smartphone OSes

**Delta Updates**
- Only transmit changed portions (not full image)
- Reduces bandwidth (100MB vs. 5GB)
- Requires binary diff algorithms (bsdiff, xdelta)

**Component-Based Updates**
- Update individual ECUs (Electronic Control Units)
- Examples: Update camera firmware, perception module, planner
- Faster, less risky than full system update

### OTA Process

**1. Update Preparation**
- Develop and test new software
- Create update package
- Sign with cryptographic key (prevents tampering)

**2. Update Campaign**
- Define target vehicles (model, year, region)
- Phased rollout: Start with small subset (1%), monitor, expand
- Schedule: Update during overnight charging (vehicle parked, connected)

**3. Download**
- Vehicle checks for updates (daily)
- Download in background (doesn't interrupt driving)
- Verify integrity (checksum, signature)

**4. Installation**
- Requires vehicle to be parked and powered
- Install to inactive partition / ECU
- Typical duration: 15-60 minutes

**5. Verification**
- Reboot and run self-tests
- Check sensors, actuators, software health
- If tests fail → Rollback automatically

**6. Confirmation**
- Report success/failure to OTA server
- OTA server tracks fleet status

### Security Considerations

**Threats:**
- **Malicious update**: Attacker uploads compromised software
- **Downgrade attack**: Force installation of old, vulnerable version
- **Man-in-the-middle**: Intercept and modify update in transit

**Defenses:**

1. **Code Signing**
   - All updates signed with manufacturer's private key
   - Vehicle verifies signature with public key
   - Unsigned updates rejected

2. **Secure Boot**
   - Boot loader verifies kernel integrity
   - Kernel verifies application integrity
   - Chain of trust from hardware to application

3. **Encrypted Transmission**
   - TLS/HTTPS for download
   - Prevents interception and tampering

4. **Rollback Protection**
   - Prevent downgrade to vulnerable versions
   - Use monotonic counter (only allow newer versions)

5. **Audit Logging**
   - Log all update attempts
   - Forensics in case of issues

### Challenges

**1. Update Size**
- Full system images: 5-10GB
- Bandwidth cost: $1-5 per vehicle per update
- Solution: Delta updates, compression, WiFi-only option

**2. Update Failures**
- Network interruption during download
- Power loss during installation
- Solution: Resumable downloads, A/B partitions, rollback

**3. Regulatory Compliance**
- Some regions require user consent for updates
- Cybersecurity regulations (UNECE WP.29)
- Solution: User notifications, consent flows

**4. Fragmentation**
- Fleet has vehicles with different hardware/software versions
- Maintaining compatibility is complex
- Solution: Versioning, compatibility checks

**5. Safety**
- Bad update could disable vehicles or cause accidents
- Need extensive testing before deployment
- Solution: Phased rollout, simulation, HIL testing

---

## 3.1 OTA Update Statistics

**Industry Examples:**

**Tesla:**
- OTA updates since 2012
- ~10-15 major updates per year
- Features added: Autopilot improvements, Navigate on Autopilot, Sentry Mode
- Bug fixes: Phantom braking, sudden unintended acceleration mitigations

**Waymo:**
- ~Weekly software updates to AV fleet
- Improvements to perception, planning, behavior
- Tested in simulation before deployment

**GM (Super Cruise):**
- OTA updates for HD maps and software
- Added features: automatic lane change, extended road coverage

**Benefits Observed:**
- **Cost savings**: $1000+ per vehicle (avoid service visits)
- **Faster improvements**: Days vs. months for traditional updates
- **Competitive advantage**: Continuous improvement vs. static product

---

## 4. Cybersecurity for Connected Vehicles

Connected and autonomous vehicles are complex cyber-physical systems vulnerable to attacks.

### Threat Landscape

**Attack Surfaces:**

1. **Wireless Interfaces**
   - Cellular (4G/5G)
   - WiFi
   - Bluetooth
   - V2X (DSRC/C-V2X)
   - Keyless entry

2. **Physical Interfaces**
   - OBD-II port
   - USB ports
   - Charging port (EVs)

3. **Backend Services**
   - OTA update server
   - Cloud APIs
   - Mobile app

4. **Supply Chain**
   - Compromised components
   - Third-party software

### Attack Examples

**2015 Jeep Cherokee Hack** (Charlie Miller & Chris Valasek)
- Remote exploit via cellular connection
- Gained control of steering, braking, acceleration
- Result: 1.4M vehicle recall

**2016 Tesla Model S Hack** (Tencent Keen Lab)
- Chained vulnerabilities in browser and CAN bus
- Remote takeover
- Result: Patched via OTA update

**2020 Tesla Key Fob Relay Attack**
- Relay signal from key fob to vehicle
- Unlock and steal vehicle
- Result: PIN-to-drive feature added

### Attack Scenarios

**1. V2X Message Spoofing**
- Attacker broadcasts fake BSM messages
- Example: "Emergency brake ahead!" causes phantom braking
- Impact: Disrupt traffic, cause collisions

**2. GPS Spoofing**
- Broadcast fake GPS signals
- Mislead vehicle about location
- Impact: Wrong route, enter restricted area

**3. Sensor Spoofing**
- Laser attack on LiDAR (overwhelm sensor)
- Adversarial examples on camera (misclassify objects)
- Impact: Fail to detect obstacles

**4. Ransomware**
- Malware locks vehicle computer
- Demand payment to unlock
- Impact: Vehicle inoperable

**5. Eavesdropping**
- Intercept V2X messages
- Track vehicle location
- Impact: Privacy violation

### Defense Strategies

**1. Network Segmentation**
- Separate safety-critical and non-critical networks
- Example: CAN bus for control, Ethernet for infotainment
- Firewall/gateway between networks

**2. Intrusion Detection Systems (IDS)**
- Monitor CAN bus for anomalous messages
- Example: Unexpected messages, high frequency, invalid values
- Alert or block suspicious traffic

**3. Message Authentication**
- Cryptographic signatures on V2X messages
- Public Key Infrastructure (PKI)
- Pseudonymous certificates (privacy-preserving)

**4. Secure Boot & Code Signing**
- Verify software integrity at boot
- Prevent execution of unauthorized code

**5. Anomaly Detection**
- Machine learning to detect unusual behavior
- Example: GPS jumps, sensor disagreements
- Trigger safe stop if attack detected

**6. Regular Updates**
- Patch vulnerabilities via OTA
- Security fixes deployed rapidly

**7. Hardware Security Modules (HSM)**
- Tamper-resistant chip stores cryptographic keys
- Cannot extract keys even with physical access

### Regulatory Standards

**UNECE WP.29 Cybersecurity Regulations** (UN R155/R156)
- Mandatory for new vehicles in EU, Japan (2022+)
- Requirements:
  - Cybersecurity Management System (CSMS)
  - Risk assessment and mitigation
  - OTA update security
  - Incident response
  - Secure by design

**ISO/SAE 21434** (Road Vehicle Cybersecurity Engineering)
- Framework for cybersecurity throughout vehicle lifecycle
- Threat analysis and risk assessment (TARA)
- Secure development processes

**NHTSA Cybersecurity Best Practices**
- US guidelines (voluntary)
- Layered security, segmentation, IDS, secure updates

### Privacy Considerations

**Data Collection:**
- Vehicles collect: Location, driving behavior, biometrics (driver monitoring)
- Who owns the data? Manufacturer? Driver? Insurance?

**Regulations:**
- **GDPR** (EU): User consent, right to deletion, data minimization
- **CCPA** (California): Right to know, delete, opt-out of sale
- **APPI** (Japan): Similar to GDPR

**Privacy-Preserving Techniques:**
- **Differential Privacy**: Add noise to data to prevent identification
- **Federated Learning**: Train models locally, only share aggregated updates
- **Pseudonymous V2X Certificates**: Change certificate frequently to prevent tracking

---

## 4.1 Cybersecurity vs. Safety

**Cybersecurity** and **Safety** are related but distinct:

| Aspect | Safety | Cybersecurity |
|--------|--------|---------------|
| Goal | Prevent harm from failures | Prevent harm from attacks |
| Threat | Random faults | Intentional adversaries |
| Standards | ISO 26262 | ISO/SAE 21434 |
| Methods | Redundancy, FMEA, FTA | Encryption, authentication, IDS |
| Mindset | "What can go wrong?" | "What can adversary do?" |

**Interaction:**
- Cybersecurity failure can cause safety failure (hacked brakes)
- Safety measures can help cybersecurity (anomaly detection catches attacks)
- Need to consider both in design

**Example:**
- **Safety concern**: Sensor fails → Use redundant sensor
- **Cybersecurity concern**: Attacker spoofs sensor → Detect anomaly, compare with other sensors

---

## Exercises

In [ ]:
### Exercise 1: V2X Message Authentication

Implement a simple message authentication system for V2X communication using cryptographic signatures.

**Tasks:**
1. Create a `SecureV2XNetwork` class that extends `V2XNetwork`
2. Implement message signing using a simple hash-based signature
3. Implement message verification that rejects tampered messages
4. Simulate a spoofing attack and show that it's detected

**Hints:**
- Use Python's `hashlib` for cryptographic hashing
- Each vehicle should have a unique secret key
- Message signature: `hash(message_content + secret_key)`
- Verification: Recompute hash and compare

```python
import hashlib

class SecureV2XNetwork:
    """V2X network with message authentication"""
    
    def __init__(self, max_range=300.0, latency=0.05, packet_loss=0.01):
        # TODO: Initialize network parameters
        # TODO: Create vehicle_keys dictionary to store secret keys
        pass
    
    def register_vehicle(self, vehicle_id: str, secret_key: str):
        """Register vehicle with its secret key"""
        # TODO: Store secret key for this vehicle
        pass
    
    def sign_message(self, message: V2XMessage, secret_key: str) -> Tuple[V2XMessage, str]:
        """Create signature for message"""
        # TODO: Create message content string (concatenate important fields)
        # TODO: Compute hash(message_content + secret_key)
        # TODO: Return message and signature
        pass
    
    def verify_message(self, message: V2XMessage, signature: str, sender_id: str) -> bool:
        """Verify message signature"""
        # TODO: Look up sender's secret key
        # TODO: Recompute expected signature
        # TODO: Return True if signatures match, False otherwise
        pass
    
    def broadcast_message(self, message: V2XMessage, signature: str):
        """Broadcast signed message"""
        # TODO: Add signature to message buffer
        # TODO: Include latency and packet loss simulation
        pass

# Example usage:
# network = SecureV2XNetwork()
# network.register_vehicle("V1", "secret_key_123")
# msg = V2XMessage(...)
# msg, sig = network.sign_message(msg, "secret_key_123")
# valid = network.verify_message(msg, sig, "V1")
# 
# # Simulate attack: modify message after signing
# msg.velocity = 999.0  # Tamper with message
# valid = network.verify_message(msg, sig, "V1")  # Should be False
```

---

### Exercise 2: OTA Update Simulator

Implement a simplified OTA update system with A/B partitions and rollback capability.

**Tasks:**
1. Create a `Vehicle` class with two partitions (A and B)
2. Implement `download_update()` that simulates downloading with failure probability
3. Implement `install_update()` that writes to inactive partition
4. Implement `verify_update()` that runs health checks (can fail randomly)
5. Implement `rollback()` that switches back to previous partition if verification fails
6. Simulate an OTA update campaign across a fleet of 100 vehicles

**Hints:**
- Track active partition (A or B)
- Store software version for each partition
- Simulate failures with `np.random.random() < failure_probability`
- Report success rate across fleet

```python
class Vehicle:
    """Vehicle with OTA update capability"""
    
    def __init__(self, vehicle_id: str):
        self.vehicle_id = vehicle_id
        self.partitions = {'A': {'version': '1.0.0', 'active': True},
                           'B': {'version': '1.0.0', 'active': False}}
        self.active_partition = 'A'
    
    def download_update(self, new_version: str, failure_prob=0.05) -> bool:
        """Download update (can fail due to network issues)"""
        # TODO: Simulate download with failure probability
        # TODO: Return True if successful, False otherwise
        pass
    
    def install_update(self, new_version: str) -> bool:
        """Install update to inactive partition"""
        # TODO: Determine inactive partition
        # TODO: Write new version to inactive partition
        # TODO: Return True if successful
        pass
    
    def verify_update(self, failure_prob=0.02) -> bool:
        """Run self-tests on new software"""
        # TODO: Simulate verification (can fail randomly)
        # TODO: Return True if tests pass, False otherwise
        pass
    
    def activate_update(self):
        """Switch to updated partition"""
        # TODO: Swap active partition
        # TODO: Update partition status
        pass
    
    def rollback(self):
        """Rollback to previous partition"""
        # TODO: Switch back to previous active partition
        pass
    
    def get_current_version(self) -> str:
        """Get currently running software version"""
        # TODO: Return version of active partition
        pass

# Simulate OTA campaign
def simulate_ota_campaign(fleet_size=100, new_version='2.0.0'):
    """Simulate OTA update across fleet"""
    # TODO: Create fleet of vehicles
    # TODO: For each vehicle:
    #   1. Download update
    #   2. Install to inactive partition
    #   3. Activate update
    #   4. Verify update
    #   5. If verification fails, rollback
    # TODO: Calculate and print success rate
    # TODO: Visualize results (success/failure counts, version distribution)
    pass

# simulate_ota_campaign(fleet_size=100, new_version='2.0.0')
```

---

### Exercise 3: CACC Energy Efficiency Analysis

Analyze the fuel savings from Cooperative Adaptive Cruise Control (CACC) platooning.

**Tasks:**
1. Extend the CACC simulation to include aerodynamic drag calculation
2. Calculate drag coefficient reduction from platooning
3. Compute fuel consumption based on drag and velocity
4. Compare fuel consumption with and without CACC
5. Visualize fuel savings over time and as a function of gap distance

**Hints:**
- Aerodynamic drag: `F_drag = 0.5 * ρ * C_d * A * v²`
- Drag reduction in platoon: Leader: 5%, Follower 1: 20%, Follower 2: 25%
- Power = F_drag × velocity
- Fuel consumption ∝ Power

**Physics:**
- ρ (air density) = 1.225 kg/m³
- C_d (drag coefficient) = 0.3 (typical car)
- A (frontal area) = 2.5 m²
- Fuel efficiency: ~30 MPG baseline, varies with drag

```python
def calculate_drag_force(velocity, drag_coefficient=0.3, frontal_area=2.5):
    """Calculate aerodynamic drag force"""
    # TODO: Compute drag force using formula above
    # ρ = 1.225 kg/m³
    pass

def get_platoon_drag_reduction(position_in_platoon):
    """Get drag reduction percentage based on position"""
    # TODO: Return drag reduction factor
    # Leader: 0.95 (5% reduction)
    # Follower 1: 0.80 (20% reduction)
    # Follower 2: 0.75 (25% reduction)
    # Follower 3+: 0.75 (25% reduction)
    pass

def calculate_fuel_consumption(drag_force, velocity, dt):
    """Calculate fuel consumed based on drag force"""
    # TODO: Power = drag_force * velocity (Watts)
    # TODO: Energy = Power * dt (Joules)
    # TODO: Convert to fuel volume (1 liter gasoline ≈ 34 MJ)
    pass

def compare_cacc_fuel_efficiency():
    """Compare fuel consumption with and without CACC"""
    # TODO: Run two simulations:
    #   1. Baseline: Vehicles with 2.0s gap (no platooning)
    #   2. CACC: Vehicles with 1.0s gap (platooning with drag reduction)
    # TODO: Track fuel consumption for each vehicle
    # TODO: Calculate total fuel consumed and percentage savings
    # TODO: Visualize:
    #   - Fuel consumption over time (baseline vs CACC)
    #   - Total fuel savings
    #   - Fuel savings vs gap distance
    pass

# compare_cacc_fuel_efficiency()
```

---

---

## References

### V2X Communication

1. **Chen, S., Hu, J., Shi, Y., & Zhao, L. (2020)**. "LTE-V: A TD-LTE-based V2X solution for future vehicular network." *IEEE Internet of Things Journal, 7*(8), 6696-6707.

2. **Naik, G., Choudhury, B., & Park, J. M. (2019)**. "IEEE 802.11bd & 5G NR V2X: Evolution of radio access technologies for V2X communications." *IEEE Access, 7*, 70169-70184.

3. **Molina-Masegosa, R., & Gozalvez, J. (2017)**. "LTE-V for sidelink 5G V2X vehicular communications: A new 5G technology for short-range vehicle-to-everything communications." *IEEE Vehicular Technology Magazine, 12*(4), 30-39.

4. **Arena, F., & Pau, G. (2019)**. "An overview of vehicular communications." *Future Internet, 11*(2), 27.

5. **Kenney, J. B. (2011)**. "Dedicated short-range communications (DSRC) standards in the United States." *Proceedings of the IEEE, 99*(7), 1162-1182.

### Cooperative Driving & CACC

6. **Shladover, S. E., Su, D., & Lu, X. Y. (2012)**. "Impacts of cooperative adaptive cruise control on freeway traffic flow." *Transportation Research Record, 2324*(1), 63-70.

7. **Ploeg, J., Shukla, D. P., van de Wouw, N., & Nijmeijer, H. (2014)**. "Controller synthesis for string stability of vehicle platoons." *IEEE Transactions on Intelligent Transportation Systems, 15*(2), 854-865.

8. **Milanes, V., Shladover, S. E., Spring, J., Nowakowski, C., Kawazoe, H., & Nakamura, M. (2014)**. "Cooperative adaptive cruise control in real traffic situations." *IEEE Transactions on Intelligent Transportation Systems, 15*(1), 296-305.

9. **Naus, G. J., Vugts, R. P., Ploeg, J., van de Molengraft, M. J., & Steinbuch, M. (2010)**. "String-stable CACC design and experimental validation: A frequency-domain approach." *IEEE Transactions on Vehicular Technology, 59*(9), 4268-4279.

10. **Rajamani, R., & Shladover, S. E. (2001)**. "An experimental comparative study of autonomous and co-operative vehicle-follower control systems." *Transportation Research Part C: Emerging Technologies, 9*(1), 15-31.

### Cloud Computing for Autonomous Vehicles

11. **Liu, L., Chen, C., Pei, Q., Maharjan, S., & Zhang, Y. (2020)**. "Vehicular edge computing and networking: A survey." *Mobile Networks and Applications, 25*, 1135-1168.

12. **Kang, J., Yu, R., Huang, X., Wu, M., Maharjan, S., Xie, S., & Zhang, Y. (2019)**. "Blockchain for secure and efficient data sharing in vehicular edge computing and networks." *IEEE Internet of Things Journal, 6*(3), 4660-4670.

13. **Huang, X., Yu, R., Kang, J., & Zhang, Y. (2017)**. "Distributed reputation management for secure and efficient vehicular edge computing and networks." *IEEE Access, 5*, 25408-25420.

14. **Raza, S., Wang, S., Ahmed, M., & Anwar, M. R. (2019)**. "A survey on vehicular edge computing: Architecture, applications, technical issues, and future directions." *Wireless Communications and Mobile Computing, 2019*.

15. **Xu, X., Xue, Y., Qi, L., Yuan, Y., Zhang, X., Umer, T., & Wan, S. (2019)**. "An edge computing-enabled computation offloading method with privacy preservation for Internet of connected vehicles." *Future Generation Computer Systems, 96*, 89-100.

### HD Maps & Localization

16. **Liu, R., Wang, J., & Zhang, B. (2020)**. "High definition map for automated driving: Overview and analysis." *The Journal of Navigation, 73*(2), 324-341.

17. **Seif, H. G., & Hu, X. (2016)**. "Autonomous driving in the iCity—HD maps as a key challenge of the automotive industry." *Engineering, 2*(2), 159-162.

18. **Poggenhans, F., Pauls, J. H., Janosovits, J., Orf, S., Naumann, M., Kuhnt, F., & Mayr, M. (2018)**. "Lanelet2: A high-definition map framework for the future of automated driving." *IEEE Intelligent Transportation Systems Conference (ITSC)*, 1672-1679.

### Over-the-Air (OTA) Updates

19. **Steger, M., Karner, M., Hillebrand, J., Druml, N., Herndl, T., Eckel, J., ... & Matischek, R. (2017)**. "A security metric for structured security analysis of automotive systems." *Computer Safety, Reliability, and Security*, 255-268.

20. **Idrees, S. M., Nowostawski, M., Jameel, R., & Mourya, A. K. (2021)**. "Security aspects of blockchain technology intended for industrial applications." *Electronics, 10*(8), 951.

21. **Sheehan, B., Murphy, F., Mullins, M., & Ryan, C. (2019)**. "Connected and autonomous vehicles: A cyber-risk classification framework." *Transportation Research Part A: Policy and Practice, 124*, 523-536.

22. **Carsten, O., & Martens, M. H. (2019)**. "How can humans understand their automated cars? HMI principles, problems and solutions." *Cognition, Technology & Work, 21*(1), 3-20.

23. **Aliwa, E., Rana, O., Perera, C., & Burnap, P. (2021)**. "Cyberattacks and countermeasures for in-vehicle networks." *ACM Computing Surveys, 54*(1), 1-37.

### Cybersecurity

24. **Parkinson, S., Ward, P., Wilson, K., & Miller, J. (2017)**. "Cyber threats facing autonomous and connected vehicles: Future challenges." *IEEE Transactions on Intelligent Transportation Systems, 18*(11), 2898-2915.

25. **Petit, J., & Shladover, S. E. (2015)**. "Potential cyberattacks on automated vehicles." *IEEE Transactions on Intelligent Transportation Systems, 16*(2), 546-556.

26. **Checkoway, S., McCoy, D., Kantor, B., Anderson, D., Shacham, H., Savage, S., ... & Kohno, T. (2011)**. "Comprehensive experimental analyses of automotive attack surfaces." *USENIX Security Symposium*, 77-92.

27. **Miller, C., & Valasek, C. (2015)**. "Remote exploitation of an unaltered passenger vehicle." *Black Hat USA, 2015*, 91.

28. **Sharma, S., & Kaushik, B. (2019)**. "A survey on internet of vehicles: Applications, security issues & solutions." *Vehicular Communications, 20*, 100182.

29. **Jo, H. J., Kim, I. S., & Lee, D. H. (2018)**. "Reliable cooperative authentication for vehicular networks." *IEEE Transactions on Intelligent Transportation Systems, 19*(4), 1065-1079.

30. **Koopman, P., & Wagner, M. (2017)**. "Autonomous vehicle safety: An interdisciplinary challenge." *IEEE Intelligent Transportation Systems Magazine, 9*(1), 90-96.

### Standards & Regulations

31. **UNECE WP.29 (2021)**. "UN Regulation No. 155: Cyber security and cyber security management system." *United Nations Economic Commission for Europe*.

32. **UNECE WP.29 (2021)**. "UN Regulation No. 156: Software update and software updates management system." *United Nations Economic Commission for Europe*.

33. **ISO/SAE 21434 (2021)**. "Road vehicles — Cybersecurity engineering." *International Organization for Standardization*.

34. **SAE J3061 (2016)**. "Cybersecurity guidebook for cyber-physical vehicle systems." *SAE International*.

35. **NHTSA (2016)**. "Cybersecurity best practices for modern vehicles." *National Highway Traffic Safety Administration*.

### Privacy & Ethics

36. **Glancy, D. J. (2012)**. "Privacy in autonomous vehicles." *Santa Clara Law Review, 52*(4), 1171-1239.

37. **Schoettle, B., & Sivak, M. (2014)**. "Public opinion about self-driving vehicles in China, India, Japan, the U.S., the U.K., and Australia." *University of Michigan Transportation Research Institute*.

38. **Fraedrich, E., Heinrichs, D., Bahamonde-Birke, F. J., & Cyganski, R. (2019)**. "Autonomous driving, the built environment and policy implications." *Transportation Research Part A: Policy and Practice, 122*, 162-172.

### Technical Reports & White Papers

39. **5GAA (2019)**. "C-V2X use cases and service level requirements." *5G Automotive Association White Paper*.

40. **ETSI (2019)**. "Intelligent Transport Systems (ITS); Vehicular Communications; Basic Set of Applications." *ETSI EN 302 637-2 V1.4.1*.

---

## Summary

In this notebook, we explored:

1. **V2X Communication**
   - Five types: V2V, V2I, V2P, V2N, V2C
   - Technologies: DSRC vs. C-V2X
   - Message types: BSM, SPaT, MAP, PSM
   - Applications: CACC, platooning, intersection management

2. **Cloud Computing**
   - Three-tier architecture: Vehicle edge, fog nodes, cloud
   - Use cases: HD maps, fleet learning, diagnostics
   - Trade-offs: Latency vs. computational power

3. **Over-the-Air Updates**
   - A/B partition updates with rollback
   - Security: Code signing, secure boot, encrypted transmission
   - Phased rollout strategies

4. **Cybersecurity**
   - Attack surfaces: Wireless, physical, backend, supply chain
   - Defense strategies: Segmentation, IDS, authentication, HSM
   - Standards: UNECE WP.29, ISO/SAE 21434

**Key Takeaways:**
- Connected and cooperative driving enable capabilities beyond individual sensors
- Cloud extends vehicle computation but requires robust connectivity
- OTA updates are essential for maintaining autonomous vehicle fleets
- Cybersecurity is critical—vehicles are safety-critical cyber-physical systems
- Privacy and security must be balanced with functionality

---